In [1]:
# First make sure to install all required packages.
# You can do it by running the following command:

In [8]:
# ]add Arrow CSV DataFrames Plots Clustering Distances FreqTables

LoadError: syntax: unexpected "]"

In [3]:
# If you launched Jupyter in directory with attached Project.toml and Manifest.toml
# use below command to install required packages with fixed versions. 
# Check Project introduction for more information.

In [6]:
#] instantiate

   Installed libfdk_aac_jll ── v2.0.2+0
   Installed Fontconfig_jll ── v2.13.93+0
   Installed x265_jll ──────── v3.5.0+0
   Installed EarCut_jll ────── v2.2.3+0
   Installed PlotUtils ─────── v1.0.13
   Installed DataAPI ───────── v1.8.0
   Installed StatsBase ─────── v0.33.10
   Installed Plots ─────────── v1.21.3
   Installed HTTP ──────────── v0.9.14
   Installed Bzip2_jll ─────── v1.0.8+0
   Installed Parsers ───────── v2.0.3
   Installed WeakRefStrings ── v1.3.0
   Installed Missings ──────── v1.0.1
   Installed FilePathsBase ─── v0.9.10
   Installed GR_jll ────────── v0.59.0+0
   Installed Cairo_jll ─────── v1.16.1+0
   Installed HarfBuzz_jll ──── v2.8.1+0
   Installed libass_jll ────── v0.15.1+0
   Installed x264_jll ──────── v2021.5.5+0
   Installed FreeType2_jll ─── v2.10.4+0
   Installed MacroTools ────── v0.5.8
   Installed RecipesPipeline ─ v0.4.0
   Installed TimeZones ─────── v1.5.7
   Installed FFMPEG_jll ────── v4.4.0+0
   Installed Graphite2_jll ─── v1.3.14+0
   Insta

In [11]:
# Import required libraries
using Arrow
#using CSV
using DataFrames
using Plots
using Clustering
using Distances
using Random
using Statistics
using FreqTables

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1342
┌ Warning: Module JSON with build ID 3143813306601 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Info: Skipping precompilation since __precompile__(false). Importing Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80].
└ @ Base loading.jl:1047
┌ Info: Precompiling FreqTables [da1fdf0e-e0ff-5433-a45f-9bb5ff651cb1]
└ @ Base loading.jl:1342
┌ Warning: Module JSON with build ID 3143813306601 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1030
┌ Warning: Module JSON with build ID 3143813306601 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that 

In [13]:
# Read clean dataset from Arrow file into DataFrame
sales_norm = DataFrame(Arrow.Table("./../../data-prep/sales_norm.arrow"))

In [ ]:
# Convert DataFrame to Matrix and make sure that one product is one column in the resulting matrix
cluster_data = Matrix(sales_norm)

In [ ]:
# Calculate Euclidean distances between all points
dists = pairwise(SqEuclidean(), cluster_data)

In [ ]:
# Specify epsilons and minpts range
eps_rng = 20.0:0.5:70.0
minpts_rng = 1:6;

In [ ]:
# Calculate average silhouettes values, number of clusters and mean clusters size 
#for each combination of hyperparameters
avg_shts_m = Vector{Float64}[]
no_clust_m = Vector{Int64}[]
avg_clust_size_m = Array{Float64}[]
for m in minpts_rng
    avg_shts = Float64[]
    no_clust = Int64[]
    avg_clust_size = Float64[]
    for e in eps_rng
        clustering = dbscan(dists, e, m)
        cluster_ids = clustering.assignments
        if maximum(cluster_ids) <= 1
            silh = NaN
        else
            idx = cluster_ids .!=0
            silh = mean(silhouettes(cluster_ids[idx], dists[idx, idx]))
        end
        push!(avg_shts, silh)
        push!(no_clust, maximum(cluster_ids))
        push!(avg_clust_size, mean(clustering.counts))
    end
    push!(avg_shts_m, avg_shts)
    push!(no_clust_m, no_clust)
    push!(avg_clust_size_m, avg_clust_size)
end

In [ ]:
# Plot average silhouettes values
# Notice that some of the lines are 'punctured' - these are cases when silhouettes value can't be computed. 
# For small values of epsilon and high 'm' all observations are classified as outliers
# For big values of epsilon all observations may be classified as one cluster (with few additional outliers)
# For epsilon > 65.0 even with m=1 all observations are assigned to one cluster
plot(eps_rng, avg_shts_m, xlab = "epsilon", ylab = "Mean silhouette value", 
    labels = permutedims(["m="*string(m) for m in 1:6]), linewidth = 2)

In [ ]:
# Plot number of clusters
plot(eps_rng, no_clust_m[2:end], xlab = "epsilon", ylab = "Number of clusters", 
    labels = permutedims(["m="*string(m) for m in 2:6]), linewidth = 2)

In [ ]:
# Plot average clusters size
plot(eps_rng, avg_clust_size_m[2:end], xlab = "epsilon", ylab = "Average cluster size",
        labels = permutedims(["m="*string(m) for m in 2:6]), linewidth = 2, legend=:topleft)

DBSCAN is producing really small clusters up to 40.0 value of epsilon - we want to have few clusters we can analyze and write down the findings. Getting back to the average silhouette plot, we can se that for epsilon > 40.0 there is a spike around epsilon equal to 50.0 for m=5 or m=6. 

We'll use epsilon=50.0 and m=6 as hyperparameter values for further analysis.

In [ ]:
# Produce final DBSCAN clustering
opt_clustering = dbscan(dists, 50.0, 6)

In [ ]:
# Check the members count in each cluster
# There are 458 products considered as outliers
#observations are dispersed in high dimensional space and don't create dense areas
freqtable(opt_clustering.assignments)

In [ ]:
# Plot clusters average sales  (also include outliers as separate cluster)
# Each cluster has some distinct characteristic we should summarize for the recipients of the report
plot(hcat([mean(cluster_data[:, opt_clustering.assignments .== i], dims=2) for i in 0:2]...), 
    xlab="Week", ylab="Normalized sales", labels=[0 1 2], linewidth=2)

In [ ]:
# Plot cluster sales standard deviations
# Variability visibly differs between groups
plot(hcat([std(cluster_data[:, opt_clustering.assignments .== i], dims=2) for i in 0:2]...), 
    xlab="Week", ylab="Normalized sales", labels=[0 1 2], linewidth=2)

In [ ]:
# Load k-means assignments from Milestone 2
kmeans_assignments = parse.(Int64, readlines("kmeans_assignments.txt"));

In [ ]:
# Produce two-way table for DBSCAN and k-means assignments
# As we can see, Cluster 2 from DBSCAN consists almost only of observations from k-means Cluster 4
# Outliers and Cluster 1 from DBSCAN have no clear mapping to k-means while considering observations membership
freqtable(opt_clustering.assignments, kmeans_assignments)

In [ ]:
# Save DBSCAN assignments to the text file
open("dbscan_assignments.txt", "w") do io
  foreach(e -> println(io, e), opt_clustering.assignments)
end

**Analysis of clustering results**

Based on DBSCAN clustering evaluation we produced 2 clusters with 353 observations in total (we removed 458 products as outliers).

* Cluster 1 has slowly increasing sales until mid-year when after rapid rise we notice even bigger drop in sales. Afterwards sales value is recovering to skyrocket in last few weeks
* Cluster 2 osciliate around steady sales state until mid-year when it drops to really small sales amount. Products are recovering quite fast, but there is sudden drop in last few weeks of observed period.
* Outliers are quite stable in terms of average sales, but are still affected by mid-year drop

Variance analysis shows that variability for oultiers is stable throughout analyzed period. Cluster 2 also exposes quite stable variability and lower in value compared to outliers. Cluster 1 has suddenly changing variance during mid- and end-year spike.

We produced two distinct clusterings using `k-means` and `DBSCAN` methods. Both algorithms are struggling to extract clearly separated groups of products, but with proper evaluation and validation we gained valuable business insights. Considering results from both clustering methods we can also see similar patterns for particular clusters:
* DBSCAN Cluster 1 is mostly a subset of k-means Clusters 1 and 2
* DBSCAN Cluster 2 is almost a subset of k-means Cluster 4

Both clustering results may be relevant for product team and person with in-depth knowledge about the products may provide additional comments regarding clustering quality. 

As a final technical detail, we want to highlight that our goal was to spot distinct groups in the product base regardless of the density of these groups. DBSCAN catches dense regions well, but related products are not guaranteed to be close to one another. We consider k-means to be our main analysis technique and DBSCAN as a validation tool.